In [42]:
from collections import namedtuple
from datetime import date

import pandas as pd

DATA_FILE = "https://bites-data.s3.us-east-2.amazonaws.com/weather-ann-arbor.csv"
STATION = namedtuple("Station", "ID Date Value")

df = pd.read_csv(DATA_FILE)

In [43]:
df

,ID,Date,Element,Data_Value
0,USW00094889,2014-11-12,TMAX,22
1,USC00208972,2009-04-29,TMIN,56
2,USC00200032,2008-05-26,TMAX,278
3,USC00205563,2005-11-11,TMAX,139
4,USC00200230,2014-02-27,TMAX,-106
...,...,...,...,...
165080,USC00205822,2015-06-09,TMAX,256
165081,USC00205822,2009-10-06,TMAX,167
165082,USC00205050,2014-07-14,TMAX,283
165083,USC00200230,2006-11-29,TMIN,117


In [44]:
df['day'] = df.apply(lambda row: row.Date[5:], axis=1)
df['year'] = df.apply(lambda row: int(row.Date[:4]), axis=1)

# remove Feb 29th rows from dataframe
df.drop(df[df['day'] == '02-29'].index, inplace=True)

In [65]:
prev_mins = df[(df['year'] < 2015) & (df['Element'] == 'TMIN')]\
    .groupby(['ID', 'day'])\
    .agg({'Data_Value': 'min'})\
    .sort_values(['ID'])

curr_mins = df[(df['year'] == 2015) & (df['Element'] == 'TMIN')]\
    .groupby(['ID', 'day'])\
    .agg({'Data_Value': 'min'})\
    .sort_values(['ID'])

prev_maxs = df[(df['year'] < 2015) & (df['Element'] == 'TMAX')]\
    .groupby(['ID', 'day'])\
    .agg({'Data_Value': 'max'})\
    .sort_values(['ID'])

curr_maxs = df[(df['year'] == 2015) & (df['Element'] == 'TMAX')]\
    .groupby(['ID', 'day'])\
    .agg({'Data_Value': 'max'})\
    .sort_values(['ID'])

In [75]:
mins = pd.merge(prev_mins, curr_mins, on=['ID', 'day'], suffixes=['_prev', '_curr'], how='right')
mins

Data_Value_prev  Data_Value_curr
ID          day                                    
USC00200032 01-01             -133              -94
            09-06               72              178
            09-05               78              189
            09-04               89              178
            09-03               94              194
...                            ...              ...
USW00094889 04-28              -32              -32
            04-27              -10              -21
            04-26              -66                0
            04-24               -6              -71
            12-31             -143              -21

[6837 rows x 2 columns]

In [92]:
mins = pd.merge(prev_mins, curr_mins, on=['ID', 'day'], suffixes=['_prev', '_curr'], how='right')
min_breakers = mins[mins['Data_Value_prev'] > mins['Data_Value_curr']]
lowest = min(min_breakers.itertuples(), key=lambda x: x[2])
lowest

Pandas(Index=('USW00094889', '02-20'), Data_Value_prev=-132, Data_Value_curr=-343)

In [90]:
# lowest = min_breakers.iloc[0]
# for x in min_breakers:
#     if x['Data_Value'] < lowest['Data_Value']:
#         lowest = x

min_breakers.min()

Data_Value_prev   -267
Data_Value_curr   -343
dtype: int64

In [64]:
# split min and max values
df2 = df[(df['year'] < 2015) & (df['Element'] == 'TMIN')].groupby(['ID', 'day']).agg({'Data_Value': 'min'}).sort_values(['ID'])
df2

Data_Value
ID          day              
USC00200032 01-01        -133
            09-06          72
            09-05          78
            09-04          89
            09-03          94
...                       ...
USW00094889 04-28         -32
            04-27         -10
            04-26         -66
            04-24          -6
            12-31        -143

[8737 rows x 1 columns]

In [46]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, USC00200032 to USW00094889
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   (Data_Value, min)  24 non-null     int64
 1   (Data_Value, max)  24 non-null     int64
dtypes: int64(2)
memory usage: 576.0+ bytes


In [47]:
df3 = df[df['year'] == 2015].groupby(['ID']).agg({'Data_Value': ['min', 'max']}).sort_values(['ID'])
df3

Data_Value     
                   min  max
ID                         
USC00200032       -278  328
USC00200228       -278  333
USC00200230       -261  328
USC00201250       -306  322
USC00201502       -317  328
USC00202308       -261  328
USC00203712       -322  322
USC00205050       -306  328
USC00205451       -278  328
USC00205563       -267  328
USC00205822       -256  344
USC00207308       -283  333
USC00207312       -272  328
USC00208080       -278  339
USC00208202       -250  328
USC00208972       -322  317
USW00004848       -238  333
USW00014833       -282  322
USW00014853       -277  361
USW00094889       -343  328

In [48]:
df3['Data_Value']['min'].__class__

pandas.core.series.Series

In [57]:
df4 = pd.merge(df3, df2, on='ID', suffixes=['_2015', '_historic'], how='right')
df4

Data_Value_2015        Data_Value_historic     
                        min    max                 min  max
ID                                                         
USC00200032          -278.0  328.0                -267  383
USC00200220             NaN    NaN                -239  378
USC00200228          -278.0  333.0                -261  378
USC00200230          -261.0  328.0                -256  378
USC00200842             NaN    NaN                -289  383
USC00201250          -306.0  322.0                -289  378
USC00201502          -317.0  328.0                -272  383
USC00202308          -261.0  328.0                -261  383
USC00203712          -322.0  322.0                -278  383
USC00205050          -306.0  328.0                -278  394
USC00205450             NaN    NaN                -250  361
USC00205451          -278.0  328.0                -278  383
USC00205563          -267.0  328.0                -261  389
USC00205822          -256.0  344.0                -250  394
USC00207308          -283.0  333.0                -294  389
USC00207312          -272.0  328.0                -306  394
USC00207320             NaN    NaN                -300  389
USC00208080          -278.0  339.0                -261  389
USC00208202          -250.0  328.0                -278  383
USC00208972          -322.0  317.0                -272  372
USW00004848          -238.0  333.0                -266  406
USW00014833          -282.0  322.0                -267  383
USW00014853          -277.0  361.0                -271  406
USW00094889          -343.0  328.0                -288  389

In [51]:
df4[df4['Data_Value_2015']['min'] < df4['Data_Value_historic']['min']]['Data_Value_2015']['min']

ID
USC00200032   -278
USC00200228   -278
USC00200230   -261
USC00201250   -306
USC00201502   -317
USC00203712   -322
USC00205050   -306
USC00205563   -267
USC00205822   -256
USC00208080   -278
USC00208972   -322
USW00014833   -282
USW00014853   -277
USW00094889   -343
Name: min, dtype: int64

In [54]:
df4[df4['Data_Value_2015']['max'] > df4['Data_Value_historic']['max']]['Data_Value_2015']['max']

Series([], Name: max, dtype: int64)

In [64]:
df.loc[(df.year == 2015) & (df.ID == df2.ID & df.Data_Value < df2.Data_Value)]


AttributeError: 'DataFrame' object has no attribute 'ID'

In [ ]:


# create sets for old max and min
old_max = df.loc[(df['Element'] == 'TMAX') & (df['year'] < 2015), ['ID', 'day', 'Data_Value']].groupby(['ID', 'day']).max(['Data_Value'])
old_min = df.loc[(df['Element'] == 'TMIN') & (df['year'] < 2015), ['ID', 'day', 'Data_Value']].groupby(['ID', 'day']).min(['Data_Value'])

results = []
# join tables where 2015 is greater than max or less than min
for x in old_max.iterrows():
    # ID is [0][0] and day is [0][1]
    found = df[(df['ID'] == x[0][0]) & (df['day'] == x[0][1]) & (df['Element'] == 'TMAX') & (df['Data_Value'] > x[1]['Data_Value'])]
    if not found.empty:
        results.extend(found.head(1).values)

for x in old_min.iterrows():
    found = df[(df['ID'] == x[0][0]) & (df['day'] == x[0][1]) & (df['Element'] == 'TMIN') & (df[1]['Data_Value'] < x['Data_Value'])]
    if not found.empty:
        results.extend(found.head(1).values)

# highest = df.where(df['Element'] == 'TMAX').groupby(['ID', 'day', 'year']).max(['Data_Value'])
# lowest = df.where(df['Element'] == 'TMIN').groupby(['ID', 'day', 'year']).min(['Data_Value'])

# pre_max = df[(df['Element'] == 'TMAX') &  (df['year'] < 2015)].groupby(['ID', 'day']).max(['Data_Value'])

# TODO: Try merging instead of iterating
pd.merge(df, old_max, on=['ID', 'day', 'Element'])